# <center>Big Compute 20 Rescale Hackathon Second Place Solution</center>
## <center>Aerodynamical optimization of an airfoil shape using Rescale High Performance Computing with Evolutionary Algorithm</center>
### <div align=right>Made by Ihor Markevych</div>

In [ ]:
import scipy.optimize
import requests
import json
from rescale.porcelain import *
import re

In [ ]:
bounds = [(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1),
          (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]

In [ ]:
# file = '''
# point#   x-coord   y-coord
# 1         0        0
# 2         0        0.5
# 3         0        0
# 4         0        0
# 5         0        0
# 6         0        0
# 7         0        0
# 8         0        0
# '''

In [ ]:
# x = [float(item) for sublist in re.findall('\d + ([\d,\.]+) +([\d,\.]+)', file) for item in sublist]

In [ ]:
key = 'YOUR API KEY'

In [ ]:
file_upload = requests.post(
  'https://platform.rescale.com/api/v2/files/contents/',
  data=None,
  files={'file': open('./files/airfoil_inv.cfg','rb')},
  headers={'Authorization': 'Token ' + key} 
)

In [ ]:
def cost(x):
    with open('./files/inputs.txt', 'w') as the_file:
        the_file.write('point#   x-coord   y-coord\n')
        for i in range(len(x) // 2):
            the_file.write(str(i) + '        ' + str(x[i * 2]) + '        ' +  str(x[i * 2 + 1]) + '\n')

    input_upload = requests.post(
      'https://platform.rescale.com/api/v2/files/contents/',
      data=None,
      files={'file': open('./files/inputs.txt','rb')},
      headers={'Authorization': 'Token ' + key} 
    )

    job_setup = requests.post(
        'https://platform.rescale.com/api/v2/jobs/',
        json = { 
            "name": "Hackaton",  
            "jobanalyses": [ 
                {  
                    "command": """gen_mesh fine
                                  run_solve 8""", 
                    "analysis": { 
                        "code": "big_compute",
                        "version": "bc2020"
                        }, 
                    "hardware": { 
                        "coresPerSlot": 64, 
                        "slots": 1, 
                        "coreType": "hpc-plus"
                        },
                    "inputFiles": [ 
                        { 
                            "id": json.loads(file_upload.text)['id']
                            },
                        {
                            "id": json.loads(input_upload.text)['id']
                        }
                        ]
                    }
                ]
            },
      headers={'Authorization': 'Token ' + key} 
    )
    
#     print(job_setup.text)
    
    requests.post(
      'https://platform.rescale.com/api/v2/jobs/' + json.loads(job_setup.content)['id'] + '/submit/',
      headers={'Authorization': 'Token ' + key} 
    )

    while json.loads(requests.get(
      'https://platform.rescale.com/api/v2/jobs/' + json.loads(job_setup.content)['id'] + '/statuses/',
      headers={'Authorization': 'Token ' + key} 
    ).text)['results'][0]['status'] != 'Completed':
        ;
       
    files = json.loads(requests.get(
        'https://platform.rescale.com/api/v2/jobs/' + json.loads(job_setup.content)['id'] + '/files/',
        headers={'Authorization': 'Token ' + key}
    ).text)['results']
    
    file = requests.get(
        'https://platform.rescale.com/api/v2/files/' + files[[x['name'] for x in files].index('output.txt')]['id'] + '/lines/',
        headers={'Authorization': 'Token ' + key} 
    ).text
    
    ld = float(re.findall('Lift/Drag: (\d+\.\d+)', file)[0])
    
    print(json.loads(job_setup.content)['id'], ld)
    return -ld

In [ ]:
# cost(x)

In [ ]:
scipy.optimize.differential_evolution(cost, bounds, disp=True)